# 融合检索：结合向量与关键词搜索

实现一个融合检索系统，将语义向量搜索与基于关键词的BM25检索优势相结合。这种方法通过同时捕获概念相似性和精确关键词匹配，提升了检索质量。

-----
为什么融合检索很重要?

传统的RAG系统通常仅依赖向量搜索，但这存在局限性：

- 向量搜索擅长捕捉语义相似性，但可能遗漏精确关键词匹配
- 关键词搜索适合特定术语检索，但缺乏语义理解能力
- 不同类型的查询在不同检索方法中表现差异显著

-----
融合检索通过以下方式实现优势互补：

- 并行执行基于向量和基于关键词的检索
- 对两种方法的得分进行标准化处理
- 通过加权公式组合两者结果
- 基于综合得分对文档进行重新排序

-----
实现步骤：
- 从 PDF 文件中提取文本
- 使用 jieba 分词器对文本进行分词，并创建向量存储
- 使用 BM25 算法对查询进行关键词匹配
- 使用向量搜索对查询进行语义匹配
- 将两种方法的结果进行加权组合，并重新排序
- 返回最终的搜索结果


In [1]:
import fitz
import os
import re
import json
import numpy as np
from tqdm import tqdm
from openai import OpenAI
from dotenv import load_dotenv
from datetime import datetime
from rank_bm25 import BM25Okapi
from sklearn.metrics.pairwise import cosine_similarity
import jieba


load_dotenv()

True

In [2]:
client = OpenAI(
    base_url=os.getenv("LLM_BASE_URL"),
    api_key=os.getenv("LLM_API_KEY")
)
llm_model = os.getenv("LLM_MODEL_ID")
embedding_model = os.getenv("EMBEDDING_MODEL_ID")

pdf_path = "../../data/AI_Information.en.zh-CN.pdf"

## 文档处理函数

In [3]:
def extract_text_from_pdf(pdf_path):
    """
    从 PDF 文件中提取文本，并打印前 `num_chars` 个字符。

    Args:
    pdf_path (str): Path to the PDF file.

    Returns:
    str: Extracted text from the PDF.
    """
    # 打开 PDF 文件
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 初始化一个空字符串以存储提取的文本

    # Iterate through each page in the PDF
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]
        text = page.get_text("text")  # 从页面中提取文本
        all_text += text  # 将提取的文本追加到 all_text 字符串中

    return all_text  # 返回提取的文本

In [4]:
def chunk_text(text, chunk_size=800, overlap=100):
    """
    将文本分割为重叠的块。

    Args:
        text (str): 要分割的输入文本
        chunk_size (int): 每个块的字符数
        overlap (int): 块之间的字符重叠数

    Returns:
        List[Dict]: 包含文本和元数据的块字典列表
    """
    chunks = []  # 初始化一个空列表来存储块

    # 使用指定的块大小和重叠迭代文本
    for i in range(0, len(text), chunk_size - overlap):
        chunk = text[i:i + chunk_size]  # 提取指定大小的块
        if chunk:  # 确保不添加空块
            chunk_data = {
                "text": chunk,  # 块文本
                "metadata": {
                    "start_char": i,    # 文本块的起始字符索引
                    "end_char": i + len(chunk)  # 文本块的结束字符索引
                }
            }
            chunks.append(chunk_data)

    print(f"创建了 {len(chunks)} 个文本块")  # 打印创建的块数
    return chunks  # 返回块列表


In [5]:
def clean_text(text):
    """
    通过移除多余的空白字符和特殊字符来清理文本。

    Args:
        text (str): 输入文本

    Returns:
        str: 清理后的文本
    """
    # 将多个空白字符（包括换行符和制表符）替换为一个空格
    text = re.sub(r'\s+', ' ', text)

    # 修复常见的OCR问题，将制表符和换行符替换为空格
    text = text.replace('\\t', ' ')
    text = text.replace('\\n', ' ')

    # 移除开头和结尾的空白字符，并确保单词之间只有一个空格
    text = ' '.join(text.split())

    return text


## 创建向量存储

In [6]:
def create_embeddings(texts):
    """
    为给定的文本创建嵌入向量。

    Args:
        texts (str 或 List[str]): 输入文本（可以是单个字符串或字符串列表）
        # model (str): 嵌入模型名称

    返回:
        List[List[float]]: 嵌入向量列表
    """
    # 处理字符串和列表类型的输入
    input_texts = texts if isinstance(texts, list) else [texts]

    # 如果需要，按批次处理（OpenAI API 有请求限制）
    batch_size = 100
    all_embeddings = []

    # 按批次迭代输入文本
    for i in range(0, len(input_texts), batch_size):
        batch = input_texts[i:i + batch_size]  # 获取当前批次的文本

        # 为当前批次创建嵌入向量
        response = client.embeddings.create(
            model=embedding_model,
            input=batch
        )

        # 从响应中提取嵌入向量
        batch_embeddings = [item.embedding for item in response.data]
        all_embeddings.extend(batch_embeddings)  # 将批次嵌入向量添加到总列表中

    # 如果输入是单个字符串，仅返回第一个嵌入向量
    if isinstance(texts, str):
        return all_embeddings[0]

    # 否则，返回所有嵌入向量
    return all_embeddings


In [7]:
class SimpleVectorStore:
    """
    使用 NumPy 实现的简单向量存储。
    """
    def __init__(self):
        self.vectors = []  # 用于存储嵌入向量的列表
        self.texts = []  # 用于存储文本内容的列表
        self.metadata = []  # 用于存储元数据的列表

    def add_item(self, text, embedding, metadata=None):
        """
        向向量存储中添加一个项目。

        Args:
            text (str): 文本内容
            embedding (List[float]): 嵌入向量
            metadata (Dict, 可选): 额外的元数据
        """
        self.vectors.append(np.array(embedding))  # 添加嵌入向量
        self.texts.append(text)  # 添加文本内容
        self.metadata.append(metadata or {})  # 添加元数据（如果为 None，则使用空字典）

    def add_items(self, items, embeddings):
        """
        向向量存储中添加多个项目。

        Args:
            items (List[Dict]): 文本项列表
            embeddings (List[List[float]]): 嵌入向量列表
        """
        for i, (item, embedding) in enumerate(zip(items, embeddings)):
            self.add_item(
                text=item["text"],  # 从项中提取文本
                embedding=embedding,  # 使用对应的嵌入
                metadata={**item.get("metadata", {}), "index": i}  # 合并项的元数据与索引
            )

    def similarity_search_with_scores(self, query_embedding, k=5):
        """
        根据查询嵌入找到最相似的项目及其相似度分数。

        Args:
            query_embedding (List[float]): 查询嵌入向量
            k (int): 返回的结果数量

        Returns:
            List[Tuple[Dict, float]]: 最相似的前 k 个项目及其分数
        """
        if not self.vectors:
            return []  # 如果没有存储向量，则返回空列表

        # 将查询嵌入转换为 NumPy 数组
        query_vector = np.array(query_embedding)

        # 使用余弦相似度计算相似性
        similarities = []
        for i, vector in enumerate(self.vectors):
            similarity = cosine_similarity([query_vector], [vector])[0][0]  # 计算余弦相似度
            similarities.append((i, similarity))  # 添加索引和相似度分数

        # 按相似度排序（降序）
        similarities.sort(key=lambda x: x[1], reverse=True)

        # 返回前 k 个结果及其分数
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],  # 根据索引检索文本
                "metadata": self.metadata[idx],  # 根据索引检索元数据
                "similarity": float(score)  # 添加相似度分数
            })

        return results

    def get_all_documents(self):
        """
        获取存储中的所有文档。

        Returns:
            List[Dict]: 所有文档
        """
        return [{"text": text, "metadata": meta} for text, meta in zip(self.texts, self.metadata)]  # 合并文本和元数据


## BM25 检索


In [8]:
def create_bm25_index(chunks):
    """
    从给定的文本块创建 BM25 索引。

    Args:
        chunks (List[Dict]): 文本块列表

    Returns:
        BM25Okapi: BM25 索引
    """
    # 从每个块中提取文本
    texts = [chunk["text"] for chunk in chunks]

    # 按空白字符分割对每个文档进行分词
    # tokenized_docs = [text.split() for text in texts]   # 英文
    tokenized_docs = [list(jieba.cut(text)) for text in texts]  # 中文

    # 使用分词后的文档创建 BM25 索引
    bm25 = BM25Okapi(tokenized_docs)

    # 打印 BM25 索引中的文档数量
    print(f"已创建包含 {len(texts)} 个文档的 BM25 索引")

    return bm25


In [9]:
def bm25_search(bm25, chunks, query, k=5):
    """
    使用查询在 BM25 索引中进行搜索。

    Args:
        bm25 (BM25Okapi): BM25 索引
        chunks (List[Dict]): 文本块列表
        query (str): 查询字符串
        k (int): 返回的结果数量

    Returns:
        List[Dict]: 带有分数的前 k 个结果
    """
    # 将查询按空格分割成单独的词
    # query_tokens = query.split()  # 英文
    query_tokens = list(jieba.cut(query))   # 中文

    # 获取查询词对已索引文档的 BM25 分数
    scores = bm25.get_scores(query_tokens)

    # 初始化一个空列表，用于存储带有分数的结果
    results = []

    # 遍历分数和对应的文本块
    for i, score in enumerate(scores):
        # 创建元数据的副本以避免修改原始数据
        metadata = chunks[i].get("metadata", {}).copy()
        # 向元数据中添加索引
        metadata["index"] = i

        results.append({
            "text": chunks[i]["text"],  # 文本内容
            "metadata": metadata,  # 带索引的元数据
            "bm25_score": float(score)  # BM25 分数
        })

    # 按 BM25 分数降序排序结果
    results.sort(key=lambda x: x["bm25_score"], reverse=True)

    # 返回前 k 个结果
    return results[:k]


## 混合检索函数

In [10]:
def fusion_retrieval(query, chunks, vector_store, bm25_index, k=5, alpha=0.5):
    """
    执行融合检索，结合基于向量和BM25的搜索。

    Args:
        query (str): 查询字符串
        chunks (List[Dict]): 原始文本块
        vector_store (SimpleVectorStore): 向量存储
        bm25_index (BM25Okapi): BM25 索引
        k (int): 返回的结果数量
        alpha (float): 向量分数的权重（0-1），其中 1-alpha 是 BM25 的权重

    Returns:
        List[Dict]: 基于综合分数的前 k 个结果
    """
    print(f"正在为查询执行融合检索: {query}")

    # 定义一个小的 epsilon 来避免除以零
    epsilon = 1e-8

    # 获取向量搜索结果
    query_embedding = create_embeddings(query)  # 为查询创建嵌入
    vector_results = vector_store.similarity_search_with_scores(query_embedding, k=len(chunks))  # 执行向量搜索

    # 获取 BM25 搜索结果
    bm25_results = bm25_search(bm25_index, chunks, query, k=len(chunks))  # 执行 BM25 搜索

    # 创建字典将文档索引映射到分数
    vector_scores_dict = {result["metadata"]["index"]: result["similarity"] for result in vector_results}
    bm25_scores_dict = {result["metadata"]["index"]: result["bm25_score"] for result in bm25_results}

    # 确保所有文档都有两种方法的分数
    all_docs = vector_store.get_all_documents()
    combined_results = []

    for i, doc in enumerate(all_docs):
        vector_score = vector_scores_dict.get(i, 0.0)  # 获取向量分数，如果未找到则为 0
        bm25_score = bm25_scores_dict.get(i, 0.0)  # 获取 BM25 分数，如果未找到则为 0
        combined_results.append({
            "text": doc["text"],
            "metadata": doc["metadata"],
            "vector_score": vector_score,
            "bm25_score": bm25_score,
            "index": i
        })

    # 提取分数为数组
    vector_scores = np.array([doc["vector_score"] for doc in combined_results])
    bm25_scores = np.array([doc["bm25_score"] for doc in combined_results])

    # 归一化分数
    norm_vector_scores = (vector_scores - np.min(vector_scores)) / (np.max(vector_scores) - np.min(vector_scores) + epsilon)
    norm_bm25_scores = (bm25_scores - np.min(bm25_scores)) / (np.max(bm25_scores) - np.min(bm25_scores) + epsilon)

    # 计算综合分数
    combined_scores = alpha * norm_vector_scores + (1 - alpha) * norm_bm25_scores

    # 将综合分数添加到结果中
    for i, score in enumerate(combined_scores):
        combined_results[i]["combined_score"] = float(score)

    # 按综合分数排序（降序）
    combined_results.sort(key=lambda x: x["combined_score"], reverse=True)

    # 返回前 k 个结果
    top_results = combined_results[:k]

    print(f"通过融合检索获取了 {len(top_results)} 份文档")
    return top_results


## 文档处理流程

In [11]:
def process_document(pdf_path, chunk_size=1000, chunk_overlap=200):
    """
    处理文档以用于融合检索。

    Args:
        pdf_path (str): PDF 文件的路径
        chunk_size (int): 每个块的大小（以字符为单位）
        chunk_overlap (int): 块之间的重叠大小（以字符为单位）

    Returns:
        Tuple[List[Dict], SimpleVectorStore, BM25Okapi]: 文本块、向量存储和 BM25 索引
    """
    # 从 PDF 文件中提取文本
    text = extract_text_from_pdf(pdf_path)

    # 清理提取的文本，去除多余的空白和特殊字符
    cleaned_text = clean_text(text)

    # 将清理后的文本分割成重叠的块
    chunks = chunk_text(cleaned_text, chunk_size, chunk_overlap)

    # 从每个块中提取文本内容以创建嵌入
    chunk_texts = [chunk["text"] for chunk in chunks]
    print("正在为文本块创建嵌入...")

    # 为文本块创建嵌入
    embeddings = create_embeddings(chunk_texts)

    # 初始化向量存储
    vector_store = SimpleVectorStore()

    # 将块及其嵌入添加到向量存储中
    vector_store.add_items(chunks, embeddings)
    print(f"已向向量存储中添加 {len(chunks)} 个项目")

    # 从块中创建 BM25 索引
    bm25_index = create_bm25_index(chunks)

    # 返回文本块、向量存储和 BM25 索引
    return chunks, vector_store, bm25_index


## 回答生成

In [12]:
def generate_response(query, context):
    """
    根据查询和上下文生成响应。

    Args:
        query (str): 用户查询
        context (str): 从检索文档中提取的上下文

    Returns:
        str: 生成的响应
    """
    # 定义系统提示，以指导AI助手
    system_prompt = """你是一个有用的AI助手。请根据提供的上下文回答用户问题。
    如果上下文中没有包含足够信息来完整回答问题，请说明这一局限性。"""

    # 使用上下文和查询格式化用户提示
    user_prompt = f"""上下文内容:
    {context}

    问题: {query}

    请根据提供的上下文回答问题。"""

    # 使用OpenAI API生成响应
    response = client.chat.completions.create(
        model=llm_model,  # 指定要使用的模型
        messages=[
            {"role": "system", "content": system_prompt},  # 系统消息，用于指导助手
            {"role": "user", "content": user_prompt}  # 用户消息，包含上下文和查询
        ],
        temperature=0.1  # 设置响应生成的温度
    )

    # 返回生成的响应
    return response.choices[0].message.content


## 主要的检索函数

In [13]:
def answer_with_fusion_rag(query, chunks, vector_store, bm25_index, k=5, alpha=0.5):
    """
    使用融合RAG方法回答查询。

    Args:
        query (str): 用户查询
        chunks (List[Dict]): 文本块列表
        vector_store (SimpleVectorStore): 向量存储
        bm25_index (BM25Okapi): BM25索引
        k (int): 检索的文档数量
        alpha (float): 向量得分的权重

    Returns:
        Dict: 包含检索到的文档和响应的查询结果
    """
    # 使用融合检索方法检索文档
    retrieved_docs = fusion_retrieval(query, chunks, vector_store, bm25_index, k=k, alpha=alpha)

    # 通过连接检索到的文档文本并使用分隔符格式化上下文
    context = "\n\n---\n\n".join([doc["text"] for doc in retrieved_docs])

    # 根据查询和格式化的上下文生成响应
    response = generate_response(query, context)

    # 返回查询、检索到的文档和生成的响应
    return {
        "query": query,
        "retrieved_documents": retrieved_docs,
        "response": response
    }


## 比较检索方法

In [14]:
def vector_only_rag(query, vector_store, k=5):
    """
    仅使用基于向量的RAG回答查询。

    Args:
        query (str): 用户查询
        vector_store (SimpleVectorStore): 向量存储
        k (int): 检索的文档数量

    Returns:
        Dict: 查询结果
    """
    # 创建查询嵌入
    query_embedding = create_embeddings(query)

    # 使用基于向量的相似性搜索检索文档
    retrieved_docs = vector_store.similarity_search_with_scores(query_embedding, k=k)

    # 通过连接检索到的文档文本并使用分隔符格式化上下文
    context = "\n\n---\n\n".join([doc["text"] for doc in retrieved_docs])

    # 根据查询和格式化的上下文生成响应
    response = generate_response(query, context)

    # 返回查询、检索到的文档和生成的响应
    return {
        "query": query,
        "retrieved_documents": retrieved_docs,
        "response": response
    }


In [15]:
def bm25_only_rag(query, chunks, bm25_index, k=5):
    """
    仅使用BM25基础的RAG回答查询。

    Args:
        query (str): 用户查询
        chunks (List[Dict]): 文本块列表
        bm25_index (BM25Okapi): BM25索引
        k (int): 检索的文档数量

    Returns:
        Dict: 查询结果
    """
    # 使用BM25搜索检索文档
    retrieved_docs = bm25_search(bm25_index, chunks, query, k=k)

    # 通过连接检索到的文档文本并使用分隔符格式化上下文
    context = "\n\n---\n\n".join([doc["text"] for doc in retrieved_docs])

    # 根据查询和格式化的上下文生成响应
    response = generate_response(query, context)

    # 返回查询、检索到的文档和生成的响应
    return {
        "query": query,
        "retrieved_documents": retrieved_docs,
        "response": response
    }


## 评估函数

In [16]:
def compare_retrieval_methods(query, chunks, vector_store, bm25_index, k=5, alpha=0.5, reference_answer=None):
    """
    比较不同检索方法对查询的效果。

    Args:
        query (str): 用户查询
        chunks (List[Dict]): 文本块列表
        vector_store (SimpleVectorStore): 向量存储
        bm25_index (BM25Okapi): BM25索引
        k (int): 检索的文档数量
        alpha (float): 融合检索中向量得分的权重
        reference_answer (str, 可选): 用于比较的参考答案

    Returns:
        Dict: 比较结果
    """
    print(f"\n=== 对查询 {query} 比较检索方法 ===\n")

    # 运行仅向量RAG
    print("\n运行仅向量RAG...")
    vector_result = vector_only_rag(query, vector_store, k)

    # 运行仅BM25 RAG
    print("\n运行仅BM25 RAG...")
    bm25_result = bm25_only_rag(query, chunks, bm25_index, k)

    # 运行融合RAG
    print("\n运行融合RAG...")
    fusion_result = answer_with_fusion_rag(query, chunks, vector_store, bm25_index, k, alpha)

    # 比较来自不同检索方法的响应
    print("\n比较响应...")
    comparison = evaluate_responses(
        query,
        vector_result["response"],
        bm25_result["response"],
        fusion_result["response"],
        reference_answer
    )

    # 返回比较结果
    return {
        "query": query,
        "vector_result": vector_result,
        "bm25_result": bm25_result,
        "fusion_result": fusion_result,
        "comparison": comparison
    }


In [17]:
def evaluate_responses(query, vector_response, bm25_response, fusion_response, reference_answer=None):
    """
    评估来自不同检索方法的响应。

    Args:
        query (str): 用户查询
        vector_response (str): 仅向量RAG的响应
        bm25_response (str): 仅BM25 RAG的响应
        fusion_response (str): 融合RAG的响应
        reference_answer (str, 可选): 参考答案

    Returns:
        str: 响应的评估结果
    """
    # 为评估人员提供系统提示，指导评估过程
    system_prompt = """你是一个RAG系统评估专家。请比较以下三种不同检索方法的回答：
    1. 基于向量的检索：使用语义相似性进行文档检索
    2. BM25关键词检索：使用关键词匹配进行文档检索
    3. 融合检索：结合向量和关键词两种方法

    根据以下标准评估回答：
    - 与查询的相关性
    - 事实准确性
    - 回答的全面性
    - 表达的清晰度和连贯性"""

    # 包含查询和响应的用户提示
    user_prompt = f"""查询: {query}

    基于向量的回答内容:
    {vector_response}

    BM25关键词回答内容:
    {bm25_response}

    融合回答内容:
    {fusion_response}
    """

    # 如果提供了参考答案，则将其添加到提示中
    if reference_answer:
        user_prompt += f"""
            参考答案:
            {reference_answer}
        """

    # 在用户提示中添加详细的比较说明
    user_prompt += """
    请对这三种回答进行详细比较分析。对于此查询，哪种方法表现最佳？为什么？
    请具体说明每种方法对于这个特定查询的优缺点。
    """

    # 使用meta-llama/Llama-3.2-3B-Instruct生成评估
    response = client.chat.completions.create(
        model=llm_model,  # 指定使用的模型
        messages=[
            {"role": "system", "content": system_prompt},  # 系统消息，用于指导评估员
            {"role": "user", "content": user_prompt}  # 用户消息，包含查询和响应
        ],
        temperature=0  # 设置响应生成的温度
    )

    # 返回生成的评估内容
    return response.choices[0].message.content


## 完整的评估流程

In [18]:
def evaluate_fusion_retrieval(pdf_path, test_queries, reference_answers=None, k=5, alpha=0.5):
    """
    评估融合检索与其他方法的对比效果。

    Args:
        pdf_path (str): PDF 文件路径
        test_queries (List[str]): 测试查询列表
        reference_answers (List[str], 可选): 参考答案列表
        k (int): 检索文档的数量
        alpha (float): 融合检索中向量分数的权重

    Returns:
        Dict: 评估结果
    """
    print("=== 正在评估融合检索 ===\n")

    # 处理文档以提取文本，创建片段，并构建向量和BM25索引
    chunks, vector_store, bm25_index = process_document(pdf_path)

    # 初始化一个列表以存储每个查询的结果
    results = []

    # 遍历每个测试查询
    for i, query in enumerate(test_queries):
        print(f"\n\n=== 评估查询 {i+1}/{len(test_queries)} ===")
        print(f"查询: {query}")

        # 如果存在，获取参考答案
        reference = None
        if reference_answers and i < len(reference_answers):
            reference = reference_answers[i]

        # 对当前查询比较不同检索方法
        comparison = compare_retrieval_methods(
            query,
            chunks,
            vector_store,
            bm25_index,
            k=k,
            alpha=alpha,
            reference_answer=reference
        )

        # 将比较结果添加到结果列表中
        results.append(comparison)

        # 打印来自不同检索方法的响应
        print("\n=== 基于向量的响应 ===")
        print(comparison["vector_result"]["response"])

        print("\n=== BM25 响应 ===")
        print(comparison["bm25_result"]["response"])

        print("\n=== 融合响应 ===")
        print(comparison["fusion_result"]["response"])

        print("\n=== 比较 ===")
        print(comparison["comparison"])

    # 生成融合检索性能的整体分析
    overall_analysis = generate_overall_analysis(results)

    # 返回结果和整体分析
    return {
        "results": results,
        "overall_analysis": overall_analysis
    }


In [19]:
def generate_overall_analysis(results):
    """
    生成融合检索的整体分析。

    Args:
        results (List[Dict]): 来自查询评估的结果

    Returns:
        str: 整体分析
    """
    # 系统提示来指导评估过程
    system_prompt = """您是一位评估信息检索系统的专家。
    根据多个测试查询，提供一个整体分析来比较以下三种检索方法：
    1. 基于向量的检索（语义相似性）
    2. BM25关键词检索（关键词匹配）
    3. 融合检索（两者的组合）

    关注点：
    1. 每种方法表现最佳的查询类型
    2. 每种方法的整体优缺点
    3. 融合检索如何平衡权衡
    4. 推荐使用每种方法的情景"""

    # 创建每个查询的评估摘要
    evaluations_summary = ""
    for i, result in enumerate(results):
        evaluations_summary += f"查询 {i+1}: {result['query']}\n"
        evaluations_summary += f"比较摘要: {result['comparison'][:200]}...\n\n"

    # 用户提示包含评估摘要
    user_prompt = f"""根据以下针对 {len(results)} 个查询的不同检索方法的评估结果，
    提供一个对基于向量、BM25和融合检索方法的整体比较分析：

    {evaluations_summary}

    请提供对这些方法的全面分析，并强调融合检索在何时以及为什么能优于单独的方法。"""

    # 使用模型生成整体分析
    response = client.chat.completions.create(
        model=llm_model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    # 返回生成的分析内容
    return response.choices[0].message.content


## 评估混合检索

In [20]:
# 定义一个与AI相关的测试查询
test_queries = [
    "Transformer模型在自然语言处理中的主要应用有哪些？"  # 特定于AI的查询

]

# 可选的参考答案
reference_answers = [
    "Transformer models have revolutionized natural language processing with applications including machine translation, text summarization, question answering, sentiment analysis, and text generation. They excel at capturing long-range dependencies in text and have become the foundation for models like BERT, GPT, and T5.",
    "Transformer模型通过包括机器翻译、文本摘要、问答、情感分析和文本生成等应用，彻底改变了自然语言处理领域。它们擅长捕捉文本中的长距离依赖关系，并已成为像BERT、GPT和T5这样的模型的基础。"
]


# 设置参数
k = 5  # 要检索的文档数量
alpha = 0.5  # 向量评分权重（0.5表示向量和BM25权重相等）

# 运行评估
evaluation_results = evaluate_fusion_retrieval(
    pdf_path=pdf_path,
    test_queries=test_queries,
    reference_answers=reference_answers,
    k=k,
    alpha=alpha
)

# 打印整体分析
print("\n\n=== OVERALL ANALYSIS ===\n")
print(evaluation_results["overall_analysis"])


=== 正在评估融合检索 ===

创建了 13 个文本块
正在为文本块创建嵌入...


Building prefix dict from the default dictionary ...


已向向量存储中添加 13 个项目


Dumping model to file cache C:\Users\jrj\AppData\Local\Temp\jieba.cache
Loading model cost 0.505 seconds.
Prefix dict has been built successfully.


已创建包含 13 个文档的 BM25 索引


=== 评估查询 1/1 ===
查询: Transformer模型在自然语言处理中的主要应用有哪些？

=== 对查询 Transformer模型在自然语言处理中的主要应用有哪些？ 比较检索方法 ===


运行仅向量RAG...

运行仅BM25 RAG...

运行融合RAG...
正在为查询执行融合检索: Transformer模型在自然语言处理中的主要应用有哪些？
通过融合检索获取了 5 份文档

比较响应...

=== 基于向量的响应 ===
根据提供的上下文，Transformer模型在自然语言处理（NLP）中的主要应用包括以下领域（尽管上下文中未直接提及"Transformer"模型，但基于NLP技术的描述可推断其典型应用场景）：

1. **机器翻译**  
   上下文明确提到NLP技术应用于机器翻译，而Transformer模型（如Google的BERT、OpenAI的GPT等）已成为现代机器翻译的核心架构。

2. **文本摘要**  
   文中提到NLP技术用于生成文本摘要，Transformer通过自注意力机制能有效处理长文本的语义压缩。

3. **情感分析**  
   上下文列举NLP在情感分析中的应用，Transformer模型可精准识别文本中的情感倾向。

4. **聊天机器人**  
   NLP技术被用于客服聊天机器人，Transformer的生成能力（如GPT系列）使其能实现更自然的对话交互。

5. **内容生成**  
   虽然未直接关联NLP章节，但后文提到AI生成文章、脚本等内容，Transformer模型（如GPT-3）是当前文本生成的主流技术。

**局限性说明**：  
上下文未明确提及"Transformer"这一术语，因此上述回答是基于NLP应用领域与Transformer模型实际技术地位的合理推断。若需更具体的Transformer架构细节（如自注意力机制），需补充其他资料。

=== BM25 响应 ===
根据提供的上下文，Transformer模型在自然语言处理（NLP）中的主要应用包括以下领域（上下文直接提及或相关技术延伸）：

1. **机器翻译**  
   - 上下文明确提到NLP技术应用于机器翻译，而Transformer模型（如Go

In [21]:
print("\n\n=== RESULTS ===\n")
print(json.dumps(evaluation_results, indent=4, ensure_ascii=False))



=== RESULTS ===

{
    "results": [
        {
            "query": "Transformer模型在自然语言处理中的主要应用有哪些？",
            "vector_result": {
                "query": "Transformer模型在自然语言处理中的主要应用有哪些？",
                "retrieved_documents": [
                    {
                        "text": "nlinedoctranslator.com 强化学习 强化学习涉及训练代理在特定环境中做出决策，以最⼤化奖励。代理通过反复试验进⾏学习，并以 奖励或惩罚的形式接收反馈。这种⽅法应⽤于游戏、机器⼈技术和资源管理。 深度学习 深度学习是机器学习的⼀个⼦领域，它使⽤多层⼈⼯神经⽹络（深度神经⽹络）来分析数据。这些 ⽹络的设计灵感来源于⼈脑的结构和功能。深度学习在图像识别、⾃然语⾔处理和语⾳识别等领域 取得了重⼤突破。 卷积神经⽹络（CNN） CNN 是⼀种深度神经⽹络，尤其适⽤于处理图像和视频。它们使⽤卷积层⾃动从输⼊数据中学习 特征。CNN ⼴泛应⽤于物体检测、⾯部识别和医学图像分析。 循环神经⽹络（RNN） RNN 旨在处理序列数据，例如⽂本和时间序列。它们具有反馈连接，可使信息随时间持续存在， 因此⾮常适合语⾔翻译、语⾳识别和情感分析等任务。 ⾃然语⾔处理（NLP） ⾃然语⾔处理 (NLP) 是⼈⼯智能的⼀个分⽀，致⼒于使计算机能够理解、解释和⽣成⼈类语⾔。 NLP 技术⼴泛应⽤于聊天机器⼈、机器翻译、⽂本摘要和情感分析。 计算机视觉 计算机视觉是⼈⼯智能的⼀个领域，它使计算机能够“查看”并解读图像和视频。这涉及物体检 测、图像分割和⾯部识别等任务。计算机视觉⼴泛应⽤于⾃动驾驶汽⻋、医学成像和监控系统。 第三章：⼈⼯智能的应⽤ ⼈⼯智能的应⽤范围⾮常⼴泛，并正在不断扩展到各个⾏业和领域。这些应⽤包括： 卫⽣保健 ⼈⼯智能正在通过医疗诊断、药物研发、个性化医疗和机器⼈⼿术等应⽤改变医疗保健。⼈⼯智能 ⼯具可以分析医学图像、预测患者预后并协助制定治疗计划。 ⾦融 在⾦融领域，⼈⼯智能⽤于欺诈检测、算法